## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-03-02-22-00 +0000,wsj,China Manufacturing Gauge Signals Weaker Growt...,https://www.wsj.com/economy/china-manufacturin...
1,2025-11-03-02-21-14 +0000,bbc,Hamas hands over three coffins it says contain...,https://www.bbc.com/news/articles/c3ep044jz47o...
2,2025-11-03-02-20-39 +0000,nypost,"Trump talks NYC mayoral race, government shutd...",https://nypost.com/2025/11/02/us-news/trump-ta...
3,2025-11-03-02-19-00 +0000,wsj,Singapore’s Top Real-Estate Asset Managers Mul...,https://www.wsj.com/business/deals/singapores-...
4,2025-11-03-02-00-00 +0000,wsj,The Arab World’s Last Militant Leader Is Elusi...,https://www.wsj.com/world/middle-east/abdulmal...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2338/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,23
544,train,9
416,new,9
84,first,8
268,mamdani,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
2,2025-11-03-02-20-39 +0000,nypost,"Trump talks NYC mayoral race, government shutd...",https://nypost.com/2025/11/02/us-news/trump-ta...,61
82,2025-11-02-20-00-00 +0000,wsj,The Trump administration believes it can conti...,https://www.wsj.com/politics/national-security...,60
108,2025-11-02-17-00-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...,57
127,2025-11-02-14-11-04 +0000,nypost,UK pol tells country to ‘get on with your live...,https://nypost.com/2025/11/02/world-news/uk-po...,51
42,2025-11-02-22-30-47 +0000,nypost,"Trump defends ICE raids, declares ‘they haven’...",https://nypost.com/2025/11/02/us-news/trump-de...,51


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
2,61,2025-11-03-02-20-39 +0000,nypost,"Trump talks NYC mayoral race, government shutd...",https://nypost.com/2025/11/02/us-news/trump-ta...
127,51,2025-11-02-14-11-04 +0000,nypost,UK pol tells country to ‘get on with your live...,https://nypost.com/2025/11/02/world-news/uk-po...
83,39,2025-11-02-19-59-19 +0000,nypost,Ex-Prince Andrew could be dealt another humili...,https://nypost.com/2025/11/02/world-news/ex-pr...
159,38,2025-11-02-10-00-00 +0000,wsj,"Republican Jack Ciattarelli, locked in a tight...",https://www.wsj.com/politics/elections/new-jer...
108,34,2025-11-02-17-00-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...
109,31,2025-11-02-16-46-09 +0000,nypost,House Dem leader rails that Mamdani isn’t futu...,https://nypost.com/2025/11/02/us-news/house-de...
94,31,2025-11-02-18-36-44 +0000,latimes,Photos: Los Angeles celebrates Dodgers' first ...,https://www.latimes.com/california/story/2025-...
114,29,2025-11-02-16-11-31 +0000,bbc,"'No help, no food, no water': Hurricane-hit Ja...",https://www.bbc.com/news/articles/cpv11w8ez10o...
82,27,2025-11-02-20-00-00 +0000,wsj,The Trump administration believes it can conti...,https://www.wsj.com/politics/national-security...
88,27,2025-11-02-19-17-09 +0000,nypost,Gavin Newsom claims he ‘never had’ concerns ab...,https://nypost.com/2025/11/02/us-news/newsom-c...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
